In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('imdbCloned.db')
cur = con.cursor()
movies = pd.read_csv("csvfiles/movies.csv")
actors = pd.read_csv("csvfiles/actors.csv")
casting = pd.read_csv("csvfiles/casting.csv")
movies.to_sql('movies', con, index=False)
actors.to_sql('actors', con, index=False)
casting.to_sql('casting', con, index=False)

In [3]:
create_movies = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE movies RENAME TO movies_no_keys;
CREATE TABLE movies (
    id INTEGER,
    title TEXT,
    release_year INTEGER,
    rating REAL,
    director TEXT,
    runtime TEXT,
    PRIMARY KEY (id)
);
INSERT INTO movies SELECT * FROM movies_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_movies)
con.commit()

In [4]:
create_actors = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE actors RENAME TO actors_no_keys;
CREATE TABLE actors (
    id INTEGER,
    name TEXT,
    PRIMARY KEY (id)
);
INSERT INTO actors SELECT * FROM actors_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_actors)
con.commit()

In [5]:
create_casting = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE casting RENAME TO casting_no_keys;
CREATE TABLE casting (
    movie_id INTEGER,
    actor_id INTEGER,
    ord INTEGER,
    FOREIGN KEY (movie_id) REFERENCES movies (id) 
            ON DELETE CASCADE ON UPDATE NO ACTION,
    FOREIGN KEY (actor_id) REFERENCES actors (id) 
            ON DELETE CASCADE ON UPDATE NO ACTION
);
INSERT INTO casting SELECT * FROM casting_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_casting)
con.commit()

In [6]:
drop_tables = """
DROP TABLE movies_no_keys;
DROP TABLE actors_no_keys;
DROP TABLE casting_no_keys;
"""
cur.executescript(drop_tables)
con.commit()

In [7]:
con.close()